In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os

MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


### Batch for Get Melodic

In [2]:
# Example passing some parameters to `func` calls. Note that you only add
# the parameters to kwargs that you need to pass. This example returns a
# list of dataframes of the melodic intervals of each piece in the corpus,
# and in this case will be chromatic and undirected intervals because of
# parameters passed in kwargs.


corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0014_3.mei',
                     'https://crimproject.org/mei/CRIM_Model_0009.mei'])
func = ImportedPiece.getMelodic  # <- NB there are no parentheses here
kwargs = {'kind': 'c', 'directed': False} 
list_of_dfs = corpus.batch(func, kwargs, metadata=True)
combined_df = pd.concat(list_of_dfs, ignore_index=True)
combined_df

Successfully imported https://crimproject.org/mei/CRIM_Mass_0014_3.mei
Successfully imported https://crimproject.org/mei/CRIM_Model_0009.mei


,Sup[erius],Contratenor,Tenor,Bassus,Composer,Title,Superius
0,Rest,NaN,Rest,Rest,Antoine de Févin,Missa Mente tota: Credo,NaN
1,Rest,0,Rest,Rest,Antoine de Févin,Missa Mente tota: Credo,NaN
2,NaN,2,NaN,NaN,Antoine de Févin,Missa Mente tota: Credo,NaN
3,NaN,2,Rest,Rest,Antoine de Févin,Missa Mente tota: Credo,NaN
4,NaN,2,NaN,NaN,Antoine de Févin,Missa Mente tota: Credo,NaN
...,...,...,...,...,...,...,...
1319,NaN,NaN,2,NaN,"Cadéac, Pierre",Je suis déshéritée,2
1320,NaN,3,1,3,"Cadéac, Pierre",Je suis déshéritée,0
1321,NaN,NaN,2,NaN,"Cadéac, Pierre",Je suis déshéritée,NaN
1322,NaN,2,2,1,"Cadéac, Pierre",Je suis déshéritée,NaN


### Here is Batch for Melodic NGrams

In [3]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0014_3.mei',
                             'https://crimproject.org/mei/CRIM_Model_0009.mei'])
func1 = ImportedPiece.getMelodic
# NB: you would probably want to set metadata to False for most preliminary results
list_of_dfs = corpus.batch(func=func1, kwargs={'end': False}, metadata=True)
func2 = ImportedPiece.getNgrams
list_of_melodic_ngrams = corpus.batch(func=func2, kwargs={'n': 4, 'df': list_of_dfs})
combined_df = pd.concat(list_of_melodic_ngrams, ignore_index=True)
combined_df

Memoized piece detected.
Memoized piece detected.


,Sup[erius],Contratenor,Tenor,Bassus,Composer,Title,Superius
0,NaN,"P1, -M2, M2, -M2",NaN,NaN,Antoine de Févin,Missa Mente tota: Credo,NaN
1,NaN,"-M2, M2, -M2, M2",NaN,NaN,Antoine de Févin,Missa Mente tota: Credo,NaN
2,NaN,"M2, -M2, M2, M2",NaN,NaN,Antoine de Févin,Missa Mente tota: Credo,NaN
3,"P1, -M2, M2, -M2","-M2, M2, M2, m2",NaN,NaN,Antoine de Févin,Missa Mente tota: Credo,NaN
4,NaN,"M2, M2, m2, -m2",NaN,NaN,Antoine de Févin,Missa Mente tota: Credo,NaN
...,...,...,...,...,...,...,...
1355,NaN,"M2, m2, M2, -M2","m2, M2, -M2, -m2",NaN,"Cadéac, Pierre",Je suis déshéritée,NaN
1356,NaN,"m2, M2, -M2, -m3","M2, -M2, -m2, -M2",NaN,"Cadéac, Pierre",Je suis déshéritée,NaN
1357,NaN,"M2, -M2, -m3, M2",NaN,NaN,"Cadéac, Pierre",Je suis déshéritée,NaN
1358,NaN,NaN,"-M2, -m2, -M2, M2",NaN,"Cadéac, Pierre",Je suis déshéritée,NaN


### Here is Batch for Cadences

In [4]:
# Example using batch to count the cadence types from multiple pieces:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0014_3.mei',
                     'https://crimproject.org/mei/CRIM_Model_0009.mei'])
list_of_dfs = corpus.batch(ImportedPiece.classifyCadences, metadata=True)
combined_df = pd.concat(list_of_dfs, ignore_index=True)
# Get the number of each type of cadence observed:
cadTypeCounts = combined_df['CadType'].value_counts()
# Get the number of cadences per Beat level:
cadTypeCounts = combined_df['Beat'].value_counts()

Memoized piece detected.
Memoized piece detected.


In [5]:
combined_df

,CadType,Evaded,LeadingTones,Low,RelLow,Tone,RelTone,Measure,Beat,Progress,SinceLast,ToNext,Composer,Title
0,Clausula Vera,False,1.0,G3,P1,G,P1,22,1.0,0.071307,168.0,64.0,Antoine de Févin,Missa Mente tota: Credo
1,Clausula Vera,False,1.0,D3,-P4,D,-P4,30,1.0,0.098472,64.0,72.0,Antoine de Févin,Missa Mente tota: Credo
2,Clausula Vera,False,1.0,G3,P1,G,P1,39,1.0,0.129032,72.0,96.0,Antoine de Févin,Missa Mente tota: Credo
3,Phrygian,False,1.0,D3,-P4,A,M2,51,1.0,0.169779,96.0,40.0,Antoine de Févin,Missa Mente tota: Credo
4,Phrygian,False,1.0,A3,M2,A,M2,56,1.0,0.186757,40.0,32.0,Antoine de Févin,Missa Mente tota: Credo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Clausula Vera,False,0.0,D3,-m3,D,-m3,15,1.0,0.455285,48.0,16.0,"Cadéac, Pierre",Je suis déshéritée
58,Clausula Vera,False,1.0,D3,-m3,D,-m3,17,1.0,0.520325,16.0,32.0,"Cadéac, Pierre",Je suis déshéritée
59,Authentic,False,1.0,F3,P1,F,P1,21,1.0,0.650407,32.0,24.0,"Cadéac, Pierre",Je suis déshéritée
60,Authentic,False,1.0,C3,-P4,C,-P4,24,1.0,0.747967,24.0,28.0,"Cadéac, Pierre",Je suis déshéritée
